In [1]:
# just in case pip will pompt for upgrade
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.1 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
!ollama --version
#Download ollama
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess
process = subprocess.Popen("ollama serve", shell=True) #runs on a different thread
#Download model
!pip install ollama
!ollama list

/bin/bash: line 1: ollama: command not found
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
NAME    ID    SIZE    MODIFIED 


In [19]:
#Download model
!ollama pull hf.co/WildBurger/group1_finetuned_gemma2_v3:Q4_K_M
# !ollama pull gemma2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 06dba9bb3e7c...   0% ▕                ▏    0 B/5.8 GB                  pulling manifest 
pulling 06dba9bb3e7c...   0% ▕                ▏    0 B/5.8 GB                  pulling manifest 
pulling 06dba9bb3e7c...   0% ▕                ▏    0 B/5.8 GB                  pulling manifest 
pulling 06dba9bb3e7c...   0% ▕                ▏ 2.0 MB/5.8 GB                  pulling manifest 
pulling 06dba9bb3e7c...   1% ▕                ▏  39 MB/5.8 GB                  pulling manifest 
pulling 06dba9bb3e7c...   2% ▕                ▏  87 MB/5.8 GB                  pulling manifest 
pulling 06dba9bb3e7c...   2% ▕                ▏ 109 MB/5.8 GB                  pulling manifest 
pulling 06dba9bb3e7c...   3% ▕                ▏ 155 MB/5.8 GB                  pulling manifest 
pulling 06dba9bb3e7c...   3% ▕                ▏ 197 MB

In [4]:
!pip install --quiet --upgrade pydantic
!pip install --quiet langchain_core langchain_ollama langchain_community
!pip install --quiet langchain_google_genai
!pip install --quiet rouge-score
!pip install --quiet seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 41.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.3 requires google-ai-generativelanguage==0.6.10, but you have google-ai-generativelanguage 0.6.17 which is incompatible.
  Preparing metadata (setup.py) ... done


In [5]:
from google import genai
from google.genai import types
import time

# from langchain_community.llms.ollama import Ollama
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

from sklearn.feature_extraction.text import TfidfVectorizer
import ollama
from sklearn.metrics import pairwise
import numpy as np
from rouge_score import rouge_scorer

from pydantic import BaseModel, Field
from typing import List, Optional
import re
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# JSON schema

class Experience(BaseModel):
    company_name : str = Field(description="The name of the company")
    role : str = Field(description="The job title or position")
    location : Optional[str] = Field(description="The location of the company")
    from_date : str = Field(description="start date of employment period")
    to_date : str = Field(description="end date of the employment")
    key_words : List[str] = Field(description="A list of technologies, skillset used in the project.")
    description : List[str] = Field(description="description of the work experience")

class Education(BaseModel):
    school : str = Field(description="name of the institution where the degree was obtained")
    from_date : str = Field(description="start date of the education")
    to_date : str = Field(description="end date of the education")
    location : str = Field(description="location of the school")
    degree : str = Field(description="Full name of the degree or the qualification")
    courses : Optional[List[str]] = Field(description="Relevant courses, subjects studied during the education")

class Project(BaseModel):
    title : str = Field(description="Name of the project")
    link : str = Field(description="Link to the project repository (convert github link)")
    key_technologies : List[str] = Field(description="A list of technologies, skillset used in the project.")
    date : str = Field(description="end date of the project")
    description : List[str] = Field(description="description of the project")

class ResumeSchema(BaseModel):
    name : str = Field(description="the full name of the candidate.")
    phone : str = Field(description="Contact phone number")
    email : str = Field(description="Contact email address")
    linkedin : str = Field(description="linkedin link")
    github : str = Field(description="github link")
    portfolio : str = Field(description="portfolio link")
    summary : str = Field(description="A brief summary highlighting key skills, expereince, and career goals.")
    skill_section : List[str] = Field(description="A list of skills from the input, work experience , projects, achivements and certifications showcasing programming languages, tools and technologies.")
    education : List[Education] = Field(description="list of educations in the input")
    work_experience : List[Experience] = Field(description="work experience")
    projects : List[Project] = Field(description="List of projects in the input")
    certifications : Optional[List[str]] = Field(description="List of certification obtained")
    achivements : List[str] = Field(description="List of achivements")

json_schema = JsonOutputParser(pydantic_object=ResumeSchema)

DATA_EXTRACTION_PROMPT_1 = '''You are tasked to fit a plain text resume in a json schema
# **JSON SCHEMA** :
- name
- phone
- email
- linkedin
- github
- portfolio
- summary
- skill_section
- education
    - school, from_date, to_date, location, degree, courses
- work_experience (work experience or job expereince)
    - company_name, role, from_date, to_date, location, description
- projects
    - title, link, key_technologies, date, description
- certifications (online courses, certifications)
- achivements

# Note:
- If certain feild is not present in the plain text, place None
- If github and linkedin only has username, convert them to url
- ensure 100% data preservation, nothing should be lost in conversion

# **PLAIN_TEXT** :
{resume_text}

# **OUTPUT**:
- plain json
- no explanation
- no code blocks
'''
DATA_EXTRACTION_PROMPT_2 = '''Input is the plain text of a resume pdf. Input: "{resume_text}". Convert this plain text to json with [User Details, Media(for social media), Summary, Educations, Skills, Work Experiences, Project, achivements, certifications]. Ensure that everything from the plain text is retained in the json. Output should be strictly a json text without any explpanations, notes, instructions or any extra text. If a any feild is missing in the plain text, replace it with None in json.'''

DATA_EXTRACTION_PROMPT_3 = '''
# Task:
    - Transform resume plain text data to a structured format.
    - The structured json information is provided under structure tag.
    - If certain section is not found then place [] (empty list) or null based on the type of the section. 
    - If there are no sub section for any section then add []. 
    - The text should be as it's in the plain text without any paraphrasing.

# json structure:
    - name
    - phone
    - email
    - linkedin
    - github
    - portfolio (link)
    - summary (text, if not present then write a paragraph which gives introduction of the resume, with highlights on projects and work experience)
    - skill_section (hard skills and soft skills preset in the resume like coding languages, social skills and technoligies etc)
    - education
        - school, from_date, to_date, location, degree, courses
    - work_experience (work experience or job expereince)
        - company_name, role, from_date, to_date, location, description
    - projects
        - title, link, key_technologies, date, description
    - certifications (online courses, certifications)
    - achivements (any explicit achivements or something achived via work or project)

# Input:
    {resume_text}

# Output:
    - A valid json string that has key's as mentioned in the structure populated with the data from the input text.
    - Ensure that there are no syntactical error with the json structure. It should follow all the explicit rules to be a valid json structure.
'''


DATA_EXTRACTION_PROMPT_4 = '''
You are an expert at extracting structured information from text. Your task is to analyze the following text, which represents a resume or professional profile, and transform it into a JSON object with the specified fields. If a field is not present in the text, leave it as null, unless otherwise specified.

**Input Text:**

{resume_text}

**Desired JSON Output Format:**

```json
{json_structure}
```
'''

JSON_STRUCTURE = '''
{
  "name": null,
  "phone": null,
  "email": null,
  "linkedin": null,
  "github": null,
  "portfolio": null,
  "summary": null,
  "skill_section": {
    "hard_skills": [],
    "soft_skills": []
  },
  "education": [
    {
      "school": null,
      "from_date": null,
      "to_date": null,
      "location": null,
      "degree": null,
      "courses": []
    }
  ],
  "work_experience": [
    {
      "company_name": null,
      "role": null,
      "from_date": null,
      "to_date": null,
      "location": null,
      "description": null
    }
  ],
  "projects": [
    {
      "title": null,
      "link": null,
      "key_technologies": [],
      "date": null,
      "description": null
    }
  ],
  "certifications": [],
  "achievements": []
}'''

In [7]:
df_base = pd.read_csv("/kaggle/input/resume-strucutred/gemma_2_v3.csv")
df_base.head()

,Unnamed: 0.1,Unnamed: 0,ID,Resume_str,Resume_html,Category,Resume_json,gemini_gen,ft_gemma2_Q8_0,ft_gemma2_Q4_K_M
0,0,0,36856210,INFORMATION TECHNOLOGY Summar...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,"{'name': 'INFORMATION TECHNOLOGY', 'summary': ...","{\n ""name"": null,\n ""email"": null,\n ""phone...","<output>""name"": null,""phone"": null,""email"": nu...","<output>\n{""name"": null, ""phone"": null, ""email..."
1,1,1,21780877,INFORMATION TECHNOLOGY SPECIALIST\tGS...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,{'name': 'INFORMATION TECHNOLOGY SPECIALIST GS...,"{\n ""name"": ""Kevin L. Trostle"",\n ""email"": n...","<output>[""name"": null, ""phone"": null, ""email"":...",NaN
2,2,2,33241454,INFORMATION TECHNOLOGY SUPERVISOR ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,"{'name': 'INFORMATION TECHNOLOGY SUPERVISOR', ...","{\n ""name"": null,\n ""email"": null,\n ""phone...","<output>\n{""resume"": [""Name"", null, ""Email"", n...","<output>[""name"": null, ""phone"": null, ""email"":..."
3,3,3,25990239,INFORMATION TECHNOLOGY INSTRUCTOR ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,"{'name': 'INFORMATION TECHNOLOGY INSTRUCTOR', ...","{\n ""name"": null,\n ""email"": null,\n ""phone...","<output>""name"": null,""phone"": null,""email"": nu...","<output>""name"": null, ""phone"": null, ""email"": ..."
4,4,4,16899268,INFORMATION TECHNOLOGY MANAGER/ANALYS...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,{'name': 'INFORMATION TECHNOLOGY MANAGER/ANALY...,"{\n ""name"": null,\n ""email"": null,\n ""phone...","<output>\n{""name"": null, ""phone"": null, ""email...","<output>[""name"": null, ""phone"": null, ""email"":..."


In [14]:
print(PromptTemplate(template=DATA_EXTRACTION_PROMPT_4, input_variables=['resume_text', 'json_structure']).format(resume_text = df_base['Resume_str'][1], json_structure = JSON_STRUCTURE))


You are an expert at extracting structured information from text. Your task is to analyze the following text, which represents a resume or professional profile, and transform it into a JSON object with the specified fields. If a field is not present in the text, leave it as null, unless otherwise specified.

**Input Text:**

         INFORMATION TECHNOLOGY SPECIALIST	GS11             Experience     07/2004   to   Current     Information Technology Specialist	GS11    Company Name   －   City  ,   State      Information Technology Specialist; Supervison; Project Management; Circuit Management; Licensed Electrician; Alarm management; Alarm Technician; Training; Supply; Quality Assurance Kevin L.  Trostle, DSN 266-4800, COMM.  865) 336-4800
Manage the assigned IT/communications environment with privileged access at the network level for the Wing, Geographically Separated Units (GSU), and Tenants.  Plan, coordinate, install, and continuously analyze system design, hardware and software.  De

In [25]:
# model = "hf.co/WildBurger/group1_finetuned_gemma2_v3:Q8_0"
model = "gemma2"
llm = ChatOllama(
    model = model,
    num_ctx = 4096 )
prompt = PromptTemplate(template=DATA_EXTRACTION_PROMPT_4, input_variables=['resume_text', 'json_structure']).format(resume_text = df_base['Resume_str'][0], json_structure = JSON_STRUCTURE)
content = llm.invoke(prompt)
print(content)

content='```json\n{\n  "name": null,\n  "phone": null,\n  "email": null,\n  "linkedin": null,\n  "github": null,\n  "portfolio": null,\n  "summary": "Dedicated Information Assurance Professional well-versed in analyzing and mitigating risk and finding cost-effective solutions. Excels at boosting performance and productivity by establishing realistic goals and enforcing deadlines. Versatile IT professional with 37 years of Enterprise design and engineering methodology.",\n  "skill_section": {\n    "hard_skills": [\n      "Enterprise platforms",\n      "Product Lifecycle Management (PLM)",\n      "Project tracking",\n      "Hardware and software upgrade planning",\n      "Product requirements documentation",\n      "MS Visio",\n      "Red Hat Enterprise Linux installation and hardening",\n      "Network Design & Troubleshooting",\n      "High Performance Computing"\n    ],\n    "soft_skills": [\n      "Self-directed",\n      "Decisive",\n      "Collaborative",\n      "Risk Management Fra

In [27]:
model = "gemma2"
llm = ChatOllama(
    model = model,
    num_ctx = 4096 )
resume_structured = []
for i in tqdm(range(df_base.shape[0])):
    prompt = PromptTemplate(template=DATA_EXTRACTION_PROMPT_4, input_variables=['resume_text', 'json_structure']).format(resume_text = df_base['Resume_str'][i], json_structure = JSON_STRUCTURE)
    content = llm.invoke(prompt)
    resume_structured.append(content.content)

100%|██████████| 120/120 [2:44:40<00:00, 82.34s/it]  


In [28]:
df_base['base_gemma'] = resume_structured
df_base.head()

,Unnamed: 0.1,Unnamed: 0,ID,Resume_str,Resume_html,Category,Resume_json,gemini_gen,ft_gemma2_Q8_0,ft_gemma2_Q4_K_M,base_gemma
0,0,0,36856210,INFORMATION TECHNOLOGY Summar...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,"{'name': 'INFORMATION TECHNOLOGY', 'summary': ...","{\n ""name"": null,\n ""email"": null,\n ""phone...","<output>""name"": null,""phone"": null,""email"": nu...","<output>\n{""name"": null, ""phone"": null, ""email...","```json\n{\n ""name"": null,\n ""phone"": null,\..."
1,1,1,21780877,INFORMATION TECHNOLOGY SPECIALIST\tGS...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,{'name': 'INFORMATION TECHNOLOGY SPECIALIST GS...,"{\n ""name"": ""Kevin L. Trostle"",\n ""email"": n...","<output>[""name"": null, ""phone"": null, ""email"":...",NaN,"```json\n{\n ""name"": ""Kevin L. Trostle"",\n ""..."
2,2,2,33241454,INFORMATION TECHNOLOGY SUPERVISOR ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,"{'name': 'INFORMATION TECHNOLOGY SUPERVISOR', ...","{\n ""name"": null,\n ""email"": null,\n ""phone...","<output>\n{""resume"": [""Name"", null, ""Email"", n...","<output>[""name"": null, ""phone"": null, ""email"":...","```json\n{\n ""name"": null,\n ""phone"": null,\..."
3,3,3,25990239,INFORMATION TECHNOLOGY INSTRUCTOR ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,"{'name': 'INFORMATION TECHNOLOGY INSTRUCTOR', ...","{\n ""name"": null,\n ""email"": null,\n ""phone...","<output>""name"": null,""phone"": null,""email"": nu...","<output>""name"": null, ""phone"": null, ""email"": ...","```json\n{\n ""name"": null,\n ""phone"": null,\..."
4,4,4,16899268,INFORMATION TECHNOLOGY MANAGER/ANALYS...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,{'name': 'INFORMATION TECHNOLOGY MANAGER/ANALY...,"{\n ""name"": null,\n ""email"": null,\n ""phone...","<output>\n{""name"": null, ""phone"": null, ""email...","<output>[""name"": null, ""phone"": null, ""email"":...","```json\n{\n ""name"": null,\n ""phone"": null,\..."


In [29]:
df_base.to_csv("gemma_extracted.csv")

In [17]:
resume_structured_q4_k_m = []
model = "hf.co/WildBurger/group1_finetuned_gemma2_v3:Q4_K_M"
llm = ChatOllama(
    model = model,
    num_ctx = 4096 )
# prompt = PromptTemplate(template=DATA_EXTRACTION_PROMPT_3, input_variables=['resume_text']).format(resume_text = df_base['Resume_str'][1])
print(llm.invoke(prompt))
for i in tqdm(range(df_base.shape[0])):
    prompt = PromptTemplate(template=DATA_EXTRACTION_PROMPT_3, input_variables=['resume_text']).format(resume_text = df_base['Resume_str'][i])
    content = llm.invoke(prompt)
    resume_structured_q4_k_m.append(content.content)

content='' additional_kwargs={} response_metadata={'model': 'hf.co/WildBurger/group1_finetuned_gemma2_v3:Q4_K_M', 'created_at': '2025-03-22T19:40:19.575290262Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10024798131, 'load_duration': 5286895590, 'prompt_eval_count': 1422, 'prompt_eval_duration': 4725658899, 'eval_count': 1, 'eval_duration': 10669158, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-710d0222-b86e-4e6a-a48c-71297de9c192-0' usage_metadata={'input_tokens': 1422, 'output_tokens': 1, 'total_tokens': 1423}


100%|██████████| 120/120 [1:53:41<00:00, 56.85s/it] 


In [18]:
df_base['ft_gemma2_Q4_K_M'] = resume_structured_q4_k_m
df_base.head()

,Unnamed: 0,ID,Resume_str,Resume_html,Category,Resume_json,gemini_gen,ft_gemma2_Q8_0,ft_gemma2_Q4_K_M
0,0,36856210,INFORMATION TECHNOLOGY Summar...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,"{'name': 'INFORMATION TECHNOLOGY', 'summary': ...","{\n ""name"": null,\n ""email"": null,\n ""phone...","<output>""name"": null,""phone"": null,""email"": nu...","<output>\n{""name"": null, ""phone"": null, ""email..."
1,1,21780877,INFORMATION TECHNOLOGY SPECIALIST\tGS...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,{'name': 'INFORMATION TECHNOLOGY SPECIALIST GS...,"{\n ""name"": ""Kevin L. Trostle"",\n ""email"": n...","<output>[""name"": null, ""phone"": null, ""email"":...",
2,2,33241454,INFORMATION TECHNOLOGY SUPERVISOR ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,"{'name': 'INFORMATION TECHNOLOGY SUPERVISOR', ...","{\n ""name"": null,\n ""email"": null,\n ""phone...","<output>\n{""resume"": [""Name"", null, ""Email"", n...","<output>[""name"": null, ""phone"": null, ""email"":..."
3,3,25990239,INFORMATION TECHNOLOGY INSTRUCTOR ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,"{'name': 'INFORMATION TECHNOLOGY INSTRUCTOR', ...","{\n ""name"": null,\n ""email"": null,\n ""phone...","<output>""name"": null,""phone"": null,""email"": nu...","<output>""name"": null, ""phone"": null, ""email"": ..."
4,4,16899268,INFORMATION TECHNOLOGY MANAGER/ANALYS...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,{'name': 'INFORMATION TECHNOLOGY MANAGER/ANALY...,"{\n ""name"": null,\n ""email"": null,\n ""phone...","<output>\n{""name"": null, ""phone"": null, ""email...","<output>[""name"": null, ""phone"": null, ""email"":..."


In [ ]:
resume_structured_q4_1 = []
model = "hf.co/WildBurger/group1_finetuned_gemma2_v3:Q4_1"
llm = ChatOllama(
    model = model,
    num_ctx = 4096 )
# prompt = PromptTemplate(template=DATA_EXTRACTION_PROMPT_3, input_variables=['resume_text']).format(resume_text = df_base['Resume_str'][1])
# print(llm.invoke(prompt))
for i in tqdm(range(df_base.shape[0])):
    prompt = PromptTemplate(template=DATA_EXTRACTION_PROMPT_3, input_variables=['resume_text']).format(resume_text = df_base['Resume_str'][i])
    content = llm.invoke(prompt)
    resume_structured_q4_1.append(content.content)
df_base['ft_gemma2_Q4_1'] = resume_structured_q4_1
df_base.head()

content='<output>\n{"name": "Kevin L. Trostle", "phone": null, "email": null, "linkedin": null, "github": null, "portfolio": null, "summary": "Information Technology Senior Manager with over 15 years of leadership experience in Information Technology as an IT Director and Consultant. Extensive strategic Vendor Management Expertise (VMO Leadership). Expert in Vendor selection process (RFI, RFP, MSA and SOW) and leader in contract negotiations. Senior Project Management leadership. Co-Chairman of Change Management Review Board. Saved Millions of Dollars in vendor expenses through successfully implemented sourcing \\"Partnerships\\". Implemented and Lead a Business Relationship Management Team. Accomplished IT Technologist with a strong Business acumen, including an MBA Degree. Successfully resolved complex Business, Technical and Operational issues. Specialist at presenting Executive Level Technical Business Presentations (VP/SVP/CIO).", "skill_section": ["Global and strategic sourcing",

 82%|████████▏ | 98/120 [1:04:10<12:25, 33.89s/it]

In [11]:
resume_structured_gemma2 = []
model = "gemma2"
llm = ChatOllama(
    model = model,
    num_ctx = 4096 )
# prompt = PromptTemplate(template=DATA_EXTRACTION_PROMPT_3, input_variables=['resume_text']).format(resume_text = df_base['Resume_str'][1])
# print(llm.invoke(prompt))
for i in tqdm(range(df_base.shape[0])):
    prompt = PromptTemplate(template=DATA_EXTRACTION_PROMPT_3, input_variables=['resume_text']).format(resume_text = df_base['Resume_str'][i])
    content = llm.invoke(prompt)
    resume_structured_gemma2.append(content.content)
df_base['base_gemma2'] = resume_structured_gemma2
df_base.head()

 26%|██▌       | 31/120 [44:32<2:07:52, 86.21s/it] 


KeyboardInterrupt: 